# About this Notebook

Bayesian probabilistic matrix factorization (BPMF) is a classical model in the recommender system field. In the following, we will discuss:

- What the BPMF is?

- How to implement BPMF mainly using Python `Numpy` with high efficiency?

- How to make data imputations with real-world spatiotemporal datasets?

If you want to know more about BPMF, please read this article:

> Ruslan Salakhutdinov, Andriy Mnih, 2008. [**Bayesian probabilistic matrix factorization using Markov chain Monte Carlo**](https://www.cs.toronto.edu/~amnih/papers/bpmf.pdf). Proceedings of the 25th International Conference on Machine Learning (*ICML 2008*), Helsinki, Finland. [[Matlab code (official)](https://www.cs.toronto.edu/~rsalakhu/BPMF.html)]

## Quick Run

This notebook is publicly available for any usage at our data imputation project. Please click [**tensor-learning**](https://github.com/xinychen/tensor-learning).


In [1]:
import numpy as np
from numpy.random import multivariate_normal as mvnrnd
from scipy.stats import wishart
from numpy.linalg import inv as inv

In [2]:
def ten2mat(tensor, mode):
    return np.reshape(np.moveaxis(tensor, mode, 0), (tensor.shape[mode], -1), order = 'F')

def mat2ten(mat, dim, mode):
    index = list()
    index.append(mode)
    for i in range(dim.shape[0]):
        if i != mode:
            index.append(i)
    return np.moveaxis(np.reshape(mat, list(dim[index]), order = 'F'), 0, mode)

In [3]:
def compute_mape(var, var_hat):
    return np.sum(np.abs(var - var_hat) / var) / var.shape[0]

def compute_rmse(var, var_hat):
    return  np.sqrt(np.sum((var - var_hat) ** 2) / var.shape[0])

# Matrix Computation Concepts

## 1) Kronecker product

- **Definition**:

Given two matrices $A\in\mathbb{R}^{m_1\times n_1}$ and $B\in\mathbb{R}^{m_2\times n_2}$, then, the **Kronecker product** between these two matrices is defined as

$$A\otimes B=\left[ \begin{array}{cccc} a_{11}B & a_{12}B & \cdots & a_{1m_2}B \\ a_{21}B & a_{22}B & \cdots & a_{2m_2}B \\ \vdots & \vdots & \ddots & \vdots \\ a_{m_11}B & a_{m_12}B & \cdots & a_{m_1m_2}B \\ \end{array} \right]$$
where the symbol $\otimes$ denotes Kronecker product, and the size of resulted $A\otimes B$ is $(m_1m_2)\times (n_1n_2)$ (i.e., $m_1\times m_2$ columns and $n_1\times n_2$ rows).

- **Example**:

If $A=\left[ \begin{array}{cc} 1 & 2 \\ 3 & 4 \\ \end{array} \right]$ and $B=\left[ \begin{array}{ccc} 5 & 6 & 7\\ 8 & 9 & 10 \\ \end{array} \right]$, then, we have

$$A\otimes B=\left[ \begin{array}{cc} 1\times \left[ \begin{array}{ccc} 5 & 6 & 7\\ 8 & 9 & 10\\ \end{array} \right] & 2\times \left[ \begin{array}{ccc} 5 & 6 & 7\\ 8 & 9 & 10\\ \end{array} \right] \\ 3\times \left[ \begin{array}{ccc} 5 & 6 & 7\\ 8 & 9 & 10\\ \end{array} \right] & 4\times \left[ \begin{array}{ccc} 5 & 6 & 7\\ 8 & 9 & 10\\ \end{array} \right] \\ \end{array} \right]$$

$$=\left[ \begin{array}{cccccc} 5 & 6 & 7 & 10 & 12 & 14 \\ 8 & 9 & 10 & 16 & 18 & 20 \\ 15 & 18 & 21 & 20 & 24 & 28 \\ 24 & 27 & 30 & 32 & 36 & 40 \\ \end{array} \right]\in\mathbb{R}^{4\times 6}.$$


## 2) Khatri-Rao product (`kr_prod`)

- **Definition**:

Given two matrices $A=\left( \boldsymbol{a}_1,\boldsymbol{a}_2,...,\boldsymbol{a}_r \right)\in\mathbb{R}^{m\times r}$ and $B=\left( \boldsymbol{b}_1,\boldsymbol{b}_2,...,\boldsymbol{b}_r \right)\in\mathbb{R}^{n\times r}$ with same number of columns, then, the **Khatri-Rao product** (or **column-wise Kronecker product**) between $A$ and $B$ is given as follows,

$$A\odot B=\left( \boldsymbol{a}_1\otimes \boldsymbol{b}_1,\boldsymbol{a}_2\otimes \boldsymbol{b}_2,...,\boldsymbol{a}_r\otimes \boldsymbol{b}_r \right)\in\mathbb{R}^{(mn)\times r}$$
where the symbol $\odot$ denotes Khatri-Rao product, and $\otimes$ denotes Kronecker product.

- **Example**:

If $A=\left[ \begin{array}{cc} 1 & 2 \\ 3 & 4 \\ \end{array} \right]=\left( \boldsymbol{a}_1,\boldsymbol{a}_2 \right) $ and $B=\left[ \begin{array}{cc} 5 & 6 \\ 7 & 8 \\ 9 & 10 \\ \end{array} \right]=\left( \boldsymbol{b}_1,\boldsymbol{b}_2 \right) $, then, we have

$$A\odot B=\left( \boldsymbol{a}_1\otimes \boldsymbol{b}_1,\boldsymbol{a}_2\otimes \boldsymbol{b}_2 \right) $$

$$=\left[ \begin{array}{cc} \left[ \begin{array}{c} 1 \\ 3 \\ \end{array} \right]\otimes \left[ \begin{array}{c} 5 \\ 7 \\ 9 \\ \end{array} \right] & \left[ \begin{array}{c} 2 \\ 4 \\ \end{array} \right]\otimes \left[ \begin{array}{c} 6 \\ 8 \\ 10 \\ \end{array} \right] \\ \end{array} \right]$$

$$=\left[ \begin{array}{cc} 5 & 12 \\ 7 & 16 \\ 9 & 20 \\ 15 & 24 \\ 21 & 32 \\ 27 & 40 \\ \end{array} \right]\in\mathbb{R}^{6\times 2}.$$

In [4]:
def kr_prod(a, b):
    return np.einsum('ir, jr -> ijr', a, b).reshape(a.shape[0] * b.shape[0], -1)

In [5]:
A = np.array([[1, 2], [3, 4]])
B = np.array([[5, 6], [7, 8], [9, 10]])
print(kr_prod(A, B))

[[ 5 12]
 [ 7 16]
 [ 9 20]
 [15 24]
 [21 32]
 [27 40]]


## 3) Computing Covariance Matrix (`cov_mat`)

For any matrix $X\in\mathbb{R}^{m\times n}$, `cov_mat` can return a $n\times n$ covariance matrix for special use in the following.

In [6]:
def cov_mat(mat):
    dim1, dim2 = mat.shape
    new_mat = np.zeros((dim2, dim2))
    mat_bar = np.mean(mat, axis = 0)
    for i in range(dim1):
        new_mat += np.einsum('i, j -> ij', mat[i, :] - mat_bar, mat[i, :] - mat_bar)
    return new_mat

# Bayesian Probabilistic Matrix Factorization (BPMF)



In [7]:
def BPMF(dense_mat, sparse_mat, init, rank, maxiter1, maxiter2):
    """Bayesian Probabilistic Matrix Factorization, BPMF."""
    W = init["W"]
    X = init["X"]
    
    dim1, dim2 = sparse_mat.shape
    dim = np.array([dim1, dim2])
    pos = np.where((dense_mat != 0) & (sparse_mat == 0))
    position = np.where(sparse_mat != 0)
    binary_mat = np.zeros((dim1, dim2))
    binary_mat[position] = 1
        
    beta0 = 1
    nu0 = rank
    mu0 = np.zeros((rank))
    W0 = np.eye(rank)
    tau = 1
    alpha = 1e-6
    beta = 1e-6
    
    W_plus = np.zeros((dim1, rank))
    X_plus = np.zeros((dim2, rank))
    mat_hat_plus = np.zeros((dim1, dim2))
    for iters in range(maxiter1):
        for order in range(2):
            if order == 0:
                mat = W.copy()
            elif order == 1:
                mat = X.copy()
            mat_bar = np.mean(mat, axis = 0)
            var_mu_hyper = (dim[order] * mat_bar + beta0 * mu0)/(dim[order] + beta0)
            var_W_hyper = inv(inv(W0) + cov_mat(mat) + dim[order] * beta0/(dim[order] + beta0)
                             * np.outer(mat_bar - mu0, mat_bar - mu0))
            var_Lambda_hyper = wishart(df = dim[order] + nu0, scale = var_W_hyper, seed = None).rvs()
            var_mu_hyper = mvnrnd(var_mu_hyper, inv((dim[order] + beta0) * var_Lambda_hyper))
            
            if order == 0:
                var1 = X.T
                mat0 = np.matmul(var1, sparse_mat.T)
            elif order == 1:
                var1 = W.T
                mat0 = np.matmul(var1, sparse_mat)
            var2 = kr_prod(var1, var1)
            if order == 0:
                mat1 = np.matmul(var2, binary_mat.T)
            elif order == 1:
                mat1 = np.matmul(var2, binary_mat)
            var3 = tau * mat1.reshape(rank, rank, dim[order]) + np.dstack([var_Lambda_hyper] * dim[order])
            var4 = tau * mat0 + np.dstack([np.matmul(var_Lambda_hyper, var_mu_hyper)] * dim[order])[0, :, :]
            for i in range(dim[order]):
                var_Lambda = var3[:, :, i]
                inv_var_Lambda = inv((var_Lambda + var_Lambda.T)/2)
                vec = mvnrnd(np.matmul(inv_var_Lambda, var4[:, i]), inv_var_Lambda)
                if order == 0:
                    W[i, :] = vec.copy()
                elif order == 1:
                    X[i, :] = vec.copy()
                    
        if iters + 1 > maxiter1 - maxiter2:
            W_plus += W
            X_plus += X
            
        mat_hat = np.matmul(W, X.T)
        if iters + 1 > maxiter1 - maxiter2:
            mat_hat_plus += mat_hat
        
        var_alpha = alpha + 0.5 * sparse_mat[position].shape[0]
        error = sparse_mat - mat_hat
        var_beta = beta + 0.5 * np.sum(error[position] ** 2)
        tau = np.random.gamma(var_alpha, 1/var_beta)

        if (iters + 1) % 1 == 0 and iters < maxiter1 - maxiter2:
            print('Iter: {}'.format(iters + 1))
            print('MAPE: {:.6}'.format(compute_mape(dense_mat[pos], mat_hat[pos])))
            print('RMSE: {:.6}'.format(compute_rmse(dense_mat[pos], mat_hat[pos])))
            print()

    W = W_plus/maxiter2
    X = X_plus/maxiter2
    mat_hat = mat_hat_plus/maxiter2
    if maxiter1 >= 1:
        print('Imputation MAPE: {:.6}'.format(compute_mape(dense_mat[pos], mat_hat[pos])))
        print('Imputation RMSE: {:.6}'.format(compute_rmse(dense_mat[pos], mat_hat[pos])))
        print()
    
    return mat_hat, W, X

# PeMS-4W

In [8]:
import numpy as np
import pandas as pd
np.random.seed(1000)

data = pd.read_csv('../datasets/California-data-set/pems-4w.csv', header = None)
dense_mat = data.values
random_mat = ten2mat(np.random.rand(data.values.shape[0], 288, 4 * 7), 0)
del data

missing_rate = 0.3

### Random missing (RM) scenario:
sparse_mat = np.multiply(dense_mat, np.round(random_mat + 0.5 - missing_rate))

In [9]:
import time
start = time.time()
dim1, dim2 = sparse_mat.shape
rank = 100
init = {"W": 0.1 * np.random.rand(dim1, rank), 
        "X": 0.1 * np.random.rand(dim2, rank)}
maxiter1 = 150
maxiter2 = 50
BPMF(dense_mat, sparse_mat, init, rank, maxiter1, maxiter2)
end = time.time()
print('Running time: %.2f minutes'%((end - start) / 60.0))

Iter: 1
MAPE: 0.0914164
RMSE: 6.58829

Iter: 2
MAPE: 0.0599643
RMSE: 4.88216

Iter: 3
MAPE: 0.0548585
RMSE: 4.62321

Iter: 4
MAPE: 0.0522293
RMSE: 4.45302

Iter: 5
MAPE: 0.0506074
RMSE: 4.34333

Iter: 6
MAPE: 0.0505103
RMSE: 4.33636

Iter: 7
MAPE: 0.0497293
RMSE: 4.27294

Iter: 8
MAPE: 0.0497112
RMSE: 4.27192

Iter: 9
MAPE: 0.0497121
RMSE: 4.27183

Iter: 10
MAPE: 0.0497092
RMSE: 4.27135

Iter: 11
MAPE: 0.0494953
RMSE: 4.25737

Iter: 12
MAPE: 0.0487945
RMSE: 4.21461

Iter: 13
MAPE: 0.0487875
RMSE: 4.21431

Iter: 14
MAPE: 0.0487917
RMSE: 4.21424

Iter: 15
MAPE: 0.048783
RMSE: 4.21389

Iter: 16
MAPE: 0.0487636
RMSE: 4.21207

Iter: 17
MAPE: 0.0482432
RMSE: 4.17036

Iter: 18
MAPE: 0.0477558
RMSE: 4.13545

Iter: 19
MAPE: 0.0476556
RMSE: 4.12711

Iter: 20
MAPE: 0.0474919
RMSE: 4.1165

Iter: 21
MAPE: 0.047009
RMSE: 4.08491

Iter: 22
MAPE: 0.0469505
RMSE: 4.08141

Iter: 23
MAPE: 0.0464782
RMSE: 4.05224

Iter: 24
MAPE: 0.046157
RMSE: 4.02948

Iter: 25
MAPE: 0.0459739
RMSE: 4.01623

Iter: 26
MAPE

In [10]:
import numpy as np
import pandas as pd
np.random.seed(1000)

data = pd.read_csv('../datasets/California-data-set/pems-4w.csv', header = None)
dense_mat = data.values
random_mat = ten2mat(np.random.rand(data.values.shape[0], 288, 4 * 7), 0)
del data

missing_rate = 0.7

### Random missing (RM) scenario:
sparse_mat = np.multiply(dense_mat, np.round(random_mat + 0.5 - missing_rate))

In [11]:
import time
start = time.time()
dim1, dim2 = sparse_mat.shape
rank = 100
init = {"W": 0.1 * np.random.rand(dim1, rank), 
        "X": 0.1 * np.random.rand(dim2, rank)}
maxiter1 = 150
maxiter2 = 50
BPMF(dense_mat, sparse_mat, init, rank, maxiter1, maxiter2)
end = time.time()
print('Running time: %.2f minutes'%((end - start) / 60.0))

Iter: 1
MAPE: 0.0930419
RMSE: 6.67999

Iter: 2
MAPE: 0.0880214
RMSE: 6.3715

Iter: 3
MAPE: 0.0674609
RMSE: 5.27922

Iter: 4
MAPE: 0.0606118
RMSE: 4.9269

Iter: 5
MAPE: 0.0605991
RMSE: 4.92554

Iter: 6
MAPE: 0.0606143
RMSE: 4.92566

Iter: 7
MAPE: 0.0606074
RMSE: 4.92559

Iter: 8
MAPE: 0.0606049
RMSE: 4.9256

Iter: 9
MAPE: 0.0606058
RMSE: 4.9255

Iter: 10
MAPE: 0.0606129
RMSE: 4.92559

Iter: 11
MAPE: 0.0606067
RMSE: 4.92561

Iter: 12
MAPE: 0.060601
RMSE: 4.92557

Iter: 13
MAPE: 0.0606075
RMSE: 4.92557

Iter: 14
MAPE: 0.0606072
RMSE: 4.92548

Iter: 15
MAPE: 0.0606057
RMSE: 4.92547

Iter: 16
MAPE: 0.0606023
RMSE: 4.92529

Iter: 17
MAPE: 0.0605936
RMSE: 4.92477

Iter: 18
MAPE: 0.060494
RMSE: 4.91874

Iter: 19
MAPE: 0.0575645
RMSE: 4.7655

Iter: 20
MAPE: 0.0568839
RMSE: 4.74158

Iter: 21
MAPE: 0.0568734
RMSE: 4.74119

Iter: 22
MAPE: 0.0568767
RMSE: 4.74121

Iter: 23
MAPE: 0.0568741
RMSE: 4.74069

Iter: 24
MAPE: 0.056867
RMSE: 4.74016

Iter: 25
MAPE: 0.0567485
RMSE: 4.73259

Iter: 26
MAPE: 0.

In [12]:
import numpy as np
import pandas as pd
np.random.seed(1000)

data = pd.read_csv('../datasets/California-data-set/pems-4w.csv', header = None)
dense_mat = data.values
dense_tensor = mat2ten(dense_mat, np.array([data.values.shape[0], 288, 4 * 7]), 0)
random_matrix = np.random.rand(data.values.shape[0], 4 * 7)

missing_rate = 0.3

### Non-random missing (NM) scenario:
binary_tensor = np.zeros(dense_tensor.shape)
for i1 in range(dense_tensor.shape[0]):
    for i2 in range(dense_tensor.shape[2]):
        binary_tensor[i1, :, i2] = np.round(random_matrix[i1, i2] + 0.5 - missing_rate)
sparse_mat = ten2mat(np.multiply(dense_tensor, binary_tensor), 0)
del dense_tensor, binary_tensor

In [13]:
import time
start = time.time()
dim1, dim2 = sparse_mat.shape
rank = 10
init = {"W": 0.1 * np.random.rand(dim1, rank), 
        "X": 0.1 * np.random.rand(dim2, rank)}
maxiter1 = 150
maxiter2 = 50
BPMF(dense_mat, sparse_mat, init, rank, maxiter1, maxiter2)
end = time.time()
print('Running time: %.2f minutes'%((end - start) / 60.0))

Iter: 1
MAPE: 0.147755
RMSE: 10.0511

Iter: 2
MAPE: 0.0734628
RMSE: 5.55547

Iter: 3
MAPE: 0.0572216
RMSE: 4.76062

Iter: 4
MAPE: 0.0549804
RMSE: 4.65352

Iter: 5
MAPE: 0.0536734
RMSE: 4.57128

Iter: 6
MAPE: 0.053152
RMSE: 4.53206

Iter: 7
MAPE: 0.0526004
RMSE: 4.49113

Iter: 8
MAPE: 0.0525178
RMSE: 4.48873

Iter: 9
MAPE: 0.052529
RMSE: 4.48955

Iter: 10
MAPE: 0.0525066
RMSE: 4.48843

Iter: 11
MAPE: 0.0524831
RMSE: 4.48648

Iter: 12
MAPE: 0.0524598
RMSE: 4.48639

Iter: 13
MAPE: 0.0524372
RMSE: 4.48489

Iter: 14
MAPE: 0.0523841
RMSE: 4.48275

Iter: 15
MAPE: 0.0523238
RMSE: 4.47969

Iter: 16
MAPE: 0.0522807
RMSE: 4.47811

Iter: 17
MAPE: 0.0522405
RMSE: 4.47653

Iter: 18
MAPE: 0.0521798
RMSE: 4.47367

Iter: 19
MAPE: 0.0521462
RMSE: 4.47244

Iter: 20
MAPE: 0.0521034
RMSE: 4.47039

Iter: 21
MAPE: 0.0520692
RMSE: 4.46883

Iter: 22
MAPE: 0.0520352
RMSE: 4.46761

Iter: 23
MAPE: 0.0520313
RMSE: 4.4678

Iter: 24
MAPE: 0.0520191
RMSE: 4.46736

Iter: 25
MAPE: 0.0519975
RMSE: 4.46627

Iter: 26
MAPE

In [14]:
import numpy as np
import pandas as pd
np.random.seed(1000)

data = pd.read_csv('../datasets/California-data-set/pems-4w.csv', header = None)
dense_mat = data.values
dense_tensor = mat2ten(dense_mat, np.array([data.values.shape[0], 288, 4 * 7]), 0)
random_matrix = np.random.rand(data.values.shape[0], 4 * 7)

missing_rate = 0.7

### Non-random missing (NM) scenario:
binary_tensor = np.zeros(dense_tensor.shape)
for i1 in range(dense_tensor.shape[0]):
    for i2 in range(dense_tensor.shape[2]):
        binary_tensor[i1, :, i2] = np.round(random_matrix[i1, i2] + 0.5 - missing_rate)
sparse_mat = ten2mat(np.multiply(dense_tensor, binary_tensor), 0)
del dense_tensor, binary_tensor

In [15]:
import time
start = time.time()
dim1, dim2 = sparse_mat.shape
rank = 10
init = {"W": 0.1 * np.random.rand(dim1, rank), 
        "X": 0.1 * np.random.rand(dim2, rank)}
maxiter1 = 150
maxiter2 = 50
BPMF(dense_mat, sparse_mat, init, rank, maxiter1, maxiter2)
end = time.time()
print('Running time: %.2f minutes'%((end - start) / 60.0))

Iter: 1
MAPE: 0.265796
RMSE: 19.1895

Iter: 2
MAPE: 0.0931291
RMSE: 6.72056

Iter: 3
MAPE: 0.074345
RMSE: 5.73687

Iter: 4
MAPE: 0.0623379
RMSE: 5.07733

Iter: 5
MAPE: 0.0603819
RMSE: 5.05779

Iter: 6
MAPE: 0.0590738
RMSE: 5.04774

Iter: 7
MAPE: 0.0589727
RMSE: 5.08512

Iter: 8
MAPE: 0.0583454
RMSE: 5.05762

Iter: 9
MAPE: 0.0583448
RMSE: 5.07966

Iter: 10
MAPE: 0.0583212
RMSE: 5.08396

Iter: 11
MAPE: 0.0582654
RMSE: 5.08069

Iter: 12
MAPE: 0.0578975
RMSE: 5.0599

Iter: 13
MAPE: 0.0578613
RMSE: 5.07036

Iter: 14
MAPE: 0.0579582
RMSE: 5.08663

Iter: 15
MAPE: 0.0580146
RMSE: 5.09696

Iter: 16
MAPE: 0.0580148
RMSE: 5.10121

Iter: 17
MAPE: 0.0580712
RMSE: 5.11037

Iter: 18
MAPE: 0.0580149
RMSE: 5.10513

Iter: 19
MAPE: 0.0580052
RMSE: 5.10662

Iter: 20
MAPE: 0.0580453
RMSE: 5.11351

Iter: 21
MAPE: 0.0580822
RMSE: 5.11621

Iter: 22
MAPE: 0.0580647
RMSE: 5.11661

Iter: 23
MAPE: 0.0581034
RMSE: 5.11879

Iter: 24
MAPE: 0.0580637
RMSE: 5.12015

Iter: 25
MAPE: 0.0580618
RMSE: 5.11783

Iter: 26
MAP

# PeMS-8W

In [16]:
import numpy as np
import pandas as pd
np.random.seed(1000)

data = pd.read_csv('../datasets/California-data-set/pems-8w.csv', header = None)
dense_mat = data.values
random_mat = ten2mat(np.random.rand(data.values.shape[0], 288, 8 * 7), 0)
del data

missing_rate = 0.3

### Random missing (RM) scenario:
sparse_mat = np.multiply(dense_mat, np.round(random_mat + 0.5 - missing_rate))

In [17]:
import time
start = time.time()
dim1, dim2 = sparse_mat.shape
rank = 100
init = {"W": 0.1 * np.random.rand(dim1, rank), 
        "X": 0.1 * np.random.rand(dim2, rank)}
maxiter1 = 150
maxiter2 = 50
BPMF(dense_mat, sparse_mat, init, rank, maxiter1, maxiter2)
end = time.time()
print('Running time: %.2f minutes'%((end - start) / 60.0))

Iter: 1
MAPE: 0.0968542
RMSE: 6.84772

Iter: 2
MAPE: 0.0601627
RMSE: 4.94326

Iter: 3
MAPE: 0.0536714
RMSE: 4.56051

Iter: 4
MAPE: 0.0519266
RMSE: 4.44854

Iter: 5
MAPE: 0.0516433
RMSE: 4.42952

Iter: 6
MAPE: 0.0510477
RMSE: 4.38518

Iter: 7
MAPE: 0.0507995
RMSE: 4.36934

Iter: 8
MAPE: 0.0504458
RMSE: 4.34578

Iter: 9
MAPE: 0.0504403
RMSE: 4.34563

Iter: 10
MAPE: 0.0504385
RMSE: 4.34536

Iter: 11
MAPE: 0.0503829
RMSE: 4.34214

Iter: 12
MAPE: 0.0499215
RMSE: 4.31329

Iter: 13
MAPE: 0.0499078
RMSE: 4.3127

Iter: 14
MAPE: 0.0498941
RMSE: 4.31214

Iter: 15
MAPE: 0.049773
RMSE: 4.30576

Iter: 16
MAPE: 0.0490059
RMSE: 4.259

Iter: 17
MAPE: 0.0488951
RMSE: 4.24898

Iter: 18
MAPE: 0.0485417
RMSE: 4.2239

Iter: 19
MAPE: 0.0485094
RMSE: 4.22119

Iter: 20
MAPE: 0.0485117
RMSE: 4.22111

Iter: 21
MAPE: 0.0485083
RMSE: 4.22088

Iter: 22
MAPE: 0.0484871
RMSE: 4.21964

Iter: 23
MAPE: 0.0482189
RMSE: 4.19844

Iter: 24
MAPE: 0.0478171
RMSE: 4.17356

Iter: 25
MAPE: 0.0477843
RMSE: 4.17131

Iter: 26
MAPE:

In [18]:
import numpy as np
import pandas as pd
np.random.seed(1000)

data = pd.read_csv('../datasets/California-data-set/pems-8w.csv', header = None)
dense_mat = data.values
random_mat = ten2mat(np.random.rand(data.values.shape[0], 288, 8 * 7), 0)
del data

missing_rate = 0.7

### Random missing (RM) scenario:
sparse_mat = np.multiply(dense_mat, np.round(random_mat + 0.5 - missing_rate))

In [19]:
import time
start = time.time()
dim1, dim2 = sparse_mat.shape
rank = 100
init = {"W": 0.1 * np.random.rand(dim1, rank), 
        "X": 0.1 * np.random.rand(dim2, rank)}
maxiter1 = 150
maxiter2 = 50
BPMF(dense_mat, sparse_mat, init, rank, maxiter1, maxiter2)
end = time.time()
print('Running time: %.2f minutes'%((end - start) / 60.0))

Iter: 1
MAPE: 0.0991596
RMSE: 6.99167

Iter: 2
MAPE: 0.0784775
RMSE: 5.82079

Iter: 3
MAPE: 0.0638669
RMSE: 5.14709

Iter: 4
MAPE: 0.0638322
RMSE: 5.14513

Iter: 5
MAPE: 0.0637671
RMSE: 5.14105

Iter: 6
MAPE: 0.060673
RMSE: 4.97382

Iter: 7
MAPE: 0.0597329
RMSE: 4.93921

Iter: 8
MAPE: 0.0597067
RMSE: 4.93872

Iter: 9
MAPE: 0.0596796
RMSE: 4.9384

Iter: 10
MAPE: 0.0596869
RMSE: 4.9384

Iter: 11
MAPE: 0.0596842
RMSE: 4.93842

Iter: 12
MAPE: 0.0596773
RMSE: 4.93827

Iter: 13
MAPE: 0.0596798
RMSE: 4.93809

Iter: 14
MAPE: 0.0596569
RMSE: 4.93683

Iter: 15
MAPE: 0.0591685
RMSE: 4.90901

Iter: 16
MAPE: 0.0574684
RMSE: 4.82021

Iter: 17
MAPE: 0.0574781
RMSE: 4.81903

Iter: 18
MAPE: 0.0574959
RMSE: 4.8188

Iter: 19
MAPE: 0.0575068
RMSE: 4.81866

Iter: 20
MAPE: 0.05752
RMSE: 4.81854

Iter: 21
MAPE: 0.0575284
RMSE: 4.81854

Iter: 22
MAPE: 0.0575317
RMSE: 4.81851

Iter: 23
MAPE: 0.0575331
RMSE: 4.81849

Iter: 24
MAPE: 0.0575396
RMSE: 4.8186

Iter: 25
MAPE: 0.0575388
RMSE: 4.81843

Iter: 26
MAPE: 0

In [20]:
import numpy as np
import pandas as pd
np.random.seed(1000)

data = pd.read_csv('../datasets/California-data-set/pems-8w.csv', header = None)
dense_mat = data.values
dense_tensor = mat2ten(dense_mat, np.array([data.values.shape[0], 288, 8 * 7]), 0)
random_matrix = np.random.rand(data.values.shape[0], 8 * 7)

missing_rate = 0.3

### Non-random missing (NM) scenario:
binary_tensor = np.zeros(dense_tensor.shape)
for i1 in range(dense_tensor.shape[0]):
    for i2 in range(dense_tensor.shape[2]):
        binary_tensor[i1, :, i2] = np.round(random_matrix[i1, i2] + 0.5 - missing_rate)
sparse_mat = ten2mat(np.multiply(dense_tensor, binary_tensor), 0)
del dense_tensor, binary_tensor

In [21]:
import time
start = time.time()
dim1, dim2 = sparse_mat.shape
rank = 10
init = {"W": 0.1 * np.random.rand(dim1, rank), 
        "X": 0.1 * np.random.rand(dim2, rank)}
maxiter1 = 150
maxiter2 = 50
BPMF(dense_mat, sparse_mat, init, rank, maxiter1, maxiter2)
end = time.time()
print('Running time: %.2f minutes'%((end - start) / 60.0))

Iter: 1
MAPE: 0.125603
RMSE: 8.38345

Iter: 2
MAPE: 0.06631
RMSE: 5.24349

Iter: 3
MAPE: 0.0593861
RMSE: 4.92073

Iter: 4
MAPE: 0.0558042
RMSE: 4.72064

Iter: 5
MAPE: 0.0549001
RMSE: 4.66421

Iter: 6
MAPE: 0.0541586
RMSE: 4.61508

Iter: 7
MAPE: 0.0534479
RMSE: 4.56849

Iter: 8
MAPE: 0.0534407
RMSE: 4.56921

Iter: 9
MAPE: 0.0534498
RMSE: 4.56964

Iter: 10
MAPE: 0.0534533
RMSE: 4.57025

Iter: 11
MAPE: 0.0534493
RMSE: 4.57011

Iter: 12
MAPE: 0.0534559
RMSE: 4.57059

Iter: 13
MAPE: 0.0534504
RMSE: 4.57045

Iter: 14
MAPE: 0.0534491
RMSE: 4.57058

Iter: 15
MAPE: 0.0534511
RMSE: 4.57092

Iter: 16
MAPE: 0.053456
RMSE: 4.57103

Iter: 17
MAPE: 0.0534497
RMSE: 4.57091

Iter: 18
MAPE: 0.0534562
RMSE: 4.57119

Iter: 19
MAPE: 0.0534501
RMSE: 4.57115

Iter: 20
MAPE: 0.0534525
RMSE: 4.57125

Iter: 21
MAPE: 0.0534554
RMSE: 4.57137

Iter: 22
MAPE: 0.0534594
RMSE: 4.57155

Iter: 23
MAPE: 0.0534548
RMSE: 4.5715

Iter: 24
MAPE: 0.0534546
RMSE: 4.57144

Iter: 25
MAPE: 0.0534554
RMSE: 4.57136

Iter: 26
MAPE:

In [22]:
import numpy as np
import pandas as pd
np.random.seed(1000)

data = pd.read_csv('../datasets/California-data-set/pems-8w.csv', header = None)
dense_mat = data.values
dense_tensor = mat2ten(dense_mat, np.array([data.values.shape[0], 288, 8 * 7]), 0)
random_matrix = np.random.rand(data.values.shape[0], 8 * 7)

missing_rate = 0.7

### Non-random missing (NM) scenario:
binary_tensor = np.zeros(dense_tensor.shape)
for i1 in range(dense_tensor.shape[0]):
    for i2 in range(dense_tensor.shape[2]):
        binary_tensor[i1, :, i2] = np.round(random_matrix[i1, i2] + 0.5 - missing_rate)
sparse_mat = ten2mat(np.multiply(dense_tensor, binary_tensor), 0)
del dense_tensor, binary_tensor

In [23]:
import time
start = time.time()
dim1, dim2 = sparse_mat.shape
rank = 10
init = {"W": 0.1 * np.random.rand(dim1, rank), 
        "X": 0.1 * np.random.rand(dim2, rank)}
maxiter1 = 150
maxiter2 = 50
BPMF(dense_mat, sparse_mat, init, rank, maxiter1, maxiter2)
end = time.time()
print('Running time: %.2f minutes'%((end - start) / 60.0))

Iter: 1
MAPE: 0.198605
RMSE: 13.6923

Iter: 2
MAPE: 0.0990871
RMSE: 7.00511

Iter: 3
MAPE: 0.0769632
RMSE: 5.8668

Iter: 4
MAPE: 0.0644758
RMSE: 5.19703

Iter: 5
MAPE: 0.0597161
RMSE: 4.95975

Iter: 6
MAPE: 0.0576576
RMSE: 4.84735

Iter: 7
MAPE: 0.0572709
RMSE: 4.82567

Iter: 8
MAPE: 0.0564141
RMSE: 4.77022

Iter: 9
MAPE: 0.0558536
RMSE: 4.73569

Iter: 10
MAPE: 0.0558808
RMSE: 4.74256

Iter: 11
MAPE: 0.0558688
RMSE: 4.7439

Iter: 12
MAPE: 0.055854
RMSE: 4.74361

Iter: 13
MAPE: 0.0558348
RMSE: 4.74315

Iter: 14
MAPE: 0.0558148
RMSE: 4.74289

Iter: 15
MAPE: 0.0557884
RMSE: 4.74127

Iter: 16
MAPE: 0.0557742
RMSE: 4.74074

Iter: 17
MAPE: 0.0557524
RMSE: 4.74027

Iter: 18
MAPE: 0.0557425
RMSE: 4.73934

Iter: 19
MAPE: 0.0557225
RMSE: 4.73864

Iter: 20
MAPE: 0.0556972
RMSE: 4.73783

Iter: 21
MAPE: 0.0556756
RMSE: 4.73621

Iter: 22
MAPE: 0.0554857
RMSE: 4.72412

Iter: 23
MAPE: 0.0552514
RMSE: 4.71115

Iter: 24
MAPE: 0.0553371
RMSE: 4.71796

Iter: 25
MAPE: 0.055372
RMSE: 4.72109

Iter: 26
MAPE: